In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]   # matplotlib setting to control the size of display images

In [8]:
def imload(impath):
    img = cv2.cvtColor(cv2.imread(impath), cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    return img, gray


def keypoint_matching(img1, img2):
    # load in the images
    img_left, gray_left = imload(img1)
    img_right, gray_right = imload(img2)

    # use SIFT to find the features
    descriptor = cv2.SIFT_create()
    kpsL, featsL = descriptor.detectAndCompute(gray_left, None)
    kpsR, featsR = descriptor.detectAndCompute(gray_right, None)

    # -- show the keypoints --
    # fig, (ax1,ax2) = plt.subplots(1,2)
    # ax1.imshow(cv2.drawKeypoints(gray_left,kpsL,None,color=(0,255,0)))
    # ax2.imshow(cv2.drawKeypoints(gray_right,kpsR,None,color=(0,255,0)))

    # using NORM_L2-- based off of euclidian distance
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    matches = matcher.match(featsL, featsR)
    sorted_matches = sorted(matches, key = lambda x:x.distance)

    # -- raise the threshold if you want more keypoints --
    thresh = 75
    good_matches = []

    maxR = 0
    maxL = 0
    
    for i in sorted_matches: 
        if i.distance < thresh:
            good_matches.append(i)
            if i.queryIdx > maxR:
                maxR = i.queryIdx
            if i.trainIdx > maxL:
                maxL = i.trainIdx

    # minimum number of matches that we will accept:
    min_matches = 10
    score = 1

    # use homography to narrow down the results
    if len(good_matches) > min_matches:
        coordsR = np.float32([kp.pt for kp in kpsR]) 
        coordsL = np.float32([kp.pt for kp in kpsL])

        # match them up by matches
        mcoordsR = np.float32([coordsR[mtch.trainIdx] for mtch in good_matches])
        mcoordsL = np.float32([coordsL[mtch.queryIdx] for mtch in good_matches])

        reprojectThresh = 4

        np.set_printoptions(precision=4, suppress=True)
        
        # use all of the points for a more accurate result
        (H, status) = cv2.findHomography(mcoordsL, mcoordsR, cv2.RANSAC, reprojectThresh)

        # drawing matches from the good_matches array-- the ones that passed the ratio test
        matched_img = cv2.drawMatches(gray_left, kpsL, gray_right, kpsR, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # -- drawing out the matches -- 
        # plt.figure()
        # plt.imshow(matched_img)
        # plt.axis('off')

        # the ratio of false positives over the number of good matches found
        score = sum(status)/len(good_matches)

    return score

In [10]:
### MAIN FUNCTION ###

# iterating through pairs of photos in the folder and pairing them up
path = "amadeus"    # the folder in which the frames are located-- change if you are looking at another folder!
count = 0
frames = sorted(os.listdir(path))
for i in range(len(frames) - 1):    
    simScore = keypoint_matching(path + "/" + frames[i], path + "/" + frames[i + 1])
    if simScore >= 1:
        count += 1
        print("there is a hard cut from " + frames[i] + " to " + frames[i+1])
    
print("there are " + str(count) + " hard cuts total")        

there is a hard cut from amadeus_00032.png to amadeus_00033.png
there is a hard cut from amadeus_00086.png to amadeus_00087.png
there is a hard cut from amadeus_00185.png to amadeus_00186.png
there is a hard cut from amadeus_00226.png to amadeus_00227.png
there is a hard cut from amadeus_00326.png to amadeus_00327.png
there is a hard cut from amadeus_00572.png to amadeus_00573.png
there are 6 hard cuts total
